# Upload Metadata to eLabFTW with API
**(author: Filippo Vasone)**


In our lab, the experimental setup is controlled by a Python program, which performs the experiments and the measures. We want to use an electronic laboratory notebook (ELN) to improve the FAIRness of the metadata gathering process. This notebook is a step towards that direction.

The chosen ELN is eLabFTW. Our aim is that the python program controlling the experiments automatically sends metadata to eLabFTW through APIs.

In this notebook I will illustrate how to upload metadata from the laboratory Python program in eLabFTW using API keys.

<u>Prerequisites</u>

Before starting, please make sure you mint an API key from the eLabFTW user interface. To do so, go to: User Panel > API key. (For further info, see: https://doc.elabftw.net/api.html)

Moreover, in this notebook I will assume that you have already created a template for your experiment (or measure) into eLabFTW from the user interface.

## 1. Getting Started with API

First of all, we need to configure the eLabFTW API.

The documentation is here: https://doc.elabftw.net/api/.

And here is the link to the Python library we will use to deal with the APIs: https://github.com/elabftw/elabapi-python.

Part of the code below is taken from the examples scripts in github: https://github.com/elabftw/elabapi-python/blob/master/examples/00-read-items.py.

In [ ]:
# install the elab library
!pip install elabapi-python

# import necessary modules
import time
import json
import elabapi_python

# replace with your api key (obtained from User Panel > API Keys)
my_api_key = '***' #(censored)

# START CONFIGURATION

configuration = elabapi_python.Configuration()
configuration.api_key['api_key'] = my_api_key
configuration.api_key_prefix['api_key'] = 'Authorization'
configuration.host = 'https://***//api/v2' # here is the server in which we installed eLabFTW (censored)
configuration.debug = False
configuration.verify_ssl = False

# create an instance of the API class
api_client = elabapi_python.ApiClient(configuration)
# fix issue with Authorization header not being properly set by the generated lib
api_client.set_default_header(header_name='Authorization', header_value=my_api_key)

# END CONFIGURATION

## 2. Getting an Experiment Template (GET)

Now we need to GET the experiment template. Please note in particular the "metadata" section, as this is the part we will update with the information coming from the experimental setup.

With the following code, we make a GET request to the server, asking for the experiment template with id = 77. This is the experiment template we have already created in eLabFTW for the SOT Experiment.

In [ ]:
# I make a GET request of the experiment template with id=77

# import necessary modules
from pprint import pprint

# create the API instance to get the experiment template

api_instance = elabapi_python.ExperimentsTemplatesApi(api_client)

id = 77 # this is the id of the experiment template

# we make the GET request of the template and print it
api_response = api_instance.get_experiment_template(id)
pprint(api_response)


## 3. Gathering Metadata

In this last section, we will actually upload the metadata onto eLabFTW. In other words, we will transform the metadata from information scattered along the Python script into a structured format using the APIs.

To do so, we will break up our task into simpler steps.

<u>Steps for the gathering of metadata</u>.

1) Create an experiment with the desired template (POST request).

2) We need the metadata to be written as variables in the code. (In this notebook, this step will be an assumption and we will just report a code snippet from the Python script controlling the experiment).

3) We need to modify the experiment we created with the desired metadata. In this step, we will use the PATCH method.

### 3.1 Create the experiment (POST)

In our first step, we have to create an experiment with the desired template (in this case the id of the template is again = 77).

In the HTTP protocol, this is the POST method. We will now perform it in the following code.

In [ ]:
## create an experiment from the desired template

# generate the EXPERIMENT instance
api_exp_instance = elabapi_python.ExperimentsApi(api_client)
# we will create the experiment with the template id=77: this information will be the input given to the method below, which creates the experiment
body = {"category_id":77}

# Create an experiment (POST)
response_from_exp_creation = api_exp_instance.post_experiment_with_http_info(body=body) # create the experiment with template id = 77

# the method 'post_experiment_with_http_info' not only creates the experiment but also gives us
# information regarding the location and id of the experiment, which is crucial to
# modify (PATCH) it in the following steps.

# the following lines of code are useful to get the location and thus the id of the newly created experiment

location = response_from_exp_creation[2].get("Location") # getting the location from the response
created_exp_id = int(location.split("/").pop()) # # getting the id from the response
print(f'The newly created experiment is here: {location}. Therefore it has the following id: {created_exp_id}')


### 3.2 Metadata from the python script

In this step we just show the variables appearing in the python script that controls our experimental setup. We want the values of these variables to become the values of the keys of the experiment metadata.

The following lines of code are copy pasted from the original script.

In [ ]:
## code snippet from the python program controlling the experimental setup

from datetime import datetime

n=datetime.now()

run_code='run11'
fielddirection='TRASV'
samplename=f'SOTefficiency/KUEP09/D6.{fielddirection}'
pulseheight=0.1  # +/- peak read current (mA)
pkvsteps=[[-0.03, 0.005], [0.005, 0.005], [0.015, 0.005], [0.03, 0.005], [0, 0.005], [-0.015, 0.005],  [-0.03, 0.005]]

#### Metadata quality
We have to modify the values coming from the Python script so that they become a human-readable input for our metadata.


In [ ]:
# a) we have to transform the string regarding field direction into a more explicit string - for this purpose we create a function

def fd_py_to_elab(fielddirection):
  """
  Transform the string regarding field direction into a more explicit string
  """
  if fielddirection == "TRASV":
    return "Transverse with regard to the current"
  elif fielddirection == "LONG":
    return "Longitudinal with regard to the current"
  else:
    return "Input not valid"

In [ ]:
# b) we have to transform the string 'samplename' in order to get only the "chip_name/pad_number" structure as wanted from the eLabFTW template
samplename_new = f'{samplename.split("/")[1]}/{samplename.split("/")[2].split(".")[0]}'
samplename_new = samplename_new[4:]

### 3.3 Modify the created experiment (PATCH)

In this final step we will modify the experiment we created with the metadata present in the python script.

To upload the metadata onto eLabFTW, we will make a PATCH request to the host server. The PATCH request modifies the experiment with the desired metadata.

In [ ]:
## modify the experiment using APIs

# what is the id of the experiment I created?
print(created_exp_id)

# I want to modify the metadata of the experiment. So first I have to look at how
# the metadata of the "empty" experiment template (id = 77) looks like.

# the following method will yield the metadata of the template as a JSON formatted string
md = api_response.metadata

# to manipulate the JSON string in python, I need to turn it into a dictionary
import json
dic_md = json.loads(md) # method that turns JSON formatted strings into a dictionary
type(dic_md) # check if everything worked
print(dic_md) # in this way I can see the structure of the dictionary to manipulate it

# add the current values of the metadata to the dictionary

dic_md['extra_fields']['Experiment_DateAndTime']['value'] = str(n) #makes it a string to avoid issues
dic_md['extra_fields']['Experiment_RunNumber']['value'] = int(run_code[3:]) #I am only interested in the run number
dic_md['extra_fields']['Experiment_MeasurementParameters_field_direction']['value'] = fd_py_to_elab(fielddirection)
dic_md['extra_fields']['UserCase_Sample_sample_name']['value'] = samplename_new
dic_md['extra_fields']['Experiment_MeasurementParameters_current_pulse']['value']= pulseheight
dic_md['extra_fields']['Experiment_MeasurementParameters_magnetic_field_sequence']['value'] = str(pkvsteps) #makes it a string (otherwise the PATCH does not work)


# check if the update of the dictionary has worked
for key in dic_md['extra_fields']:
  print(dic_md['extra_fields'][key]['value'])

#turn the dictionary into a JSON formatted string again using dumps() method (in order to PATCH the experiment)

new_md = json.dumps(dic_md)


# now I insert the json formatted string into the experiment using the PATCH method

api_exp_instance.patch_experiment_with_http_info(created_exp_id, body ={'title': 'My_SOT_experiment_from_API', 'metadata': new_md})